# LightFM

LightFMは、Pythonのためのレコメンデーションシステム構築ライブラリです。Lyst Ltdにより開発され、ハイブリッド協調フィルタリングとコンテンツベースの手法を融合していることが特徴です。
- GitHub：https://github.com/lyst/lightfm

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/recommendation/collaborative_filtering(linear)/LightFM.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install lightfm

In [4]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
data = fetch_movielens(min_rating=5.0)

# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

# Evaluate the trained model
test_precision = precision_at_k(model, data['test'], k=5).mean()

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


ImportError: dlopen(/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so, 0x0002): tried: '/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so' (no such file), '/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [2]:
from lightfm.datasets import fetch_movielens
data = fetch_movielens()

data.keys()

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


ImportError: dlopen(/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so, 0x0002): tried: '/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so' (no such file), '/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/lightfm/_lightfm_fast_no_openmp.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [ ]:
train = data["train"]
test = data["test"]

model = LightFM(no_components=10,learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
